# Import Data and Split

Run all the commented out code the first time you run the script.

In [ ]:
# !ls results
# !pip install scikit-learn


In [ ]:
# !rm -rf ./Dataset
# !rm -rf ./mimicDataset/*
# !nproc
# !rm -rf ./results

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
# zip file

# import zipfile
# import os

# # Specify the path to the ZIP file
# zip_path = 'mimicDataset/mimicDataset.zip'

# # Open the ZIP file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     # Extract all the files in the same folder
#     zip_ref.extractall()

# # Get the directory containing the ZIP file
# zip_dir = os.path.dirname(zip_path)

# # Print the list of extracted files
# extracted_files = zip_ref.namelist()
# for file in extracted_files:
#     print(os.path.join(zip_dir, file))


In [ ]:
import json
import os
import shutil
 
# set env and make folder
dataset_folder_path = 'Dataset'
mimic_dataset_path = 'mimicDataset'


if not os.path.exists(dataset_folder_path):
    os.mkdir(dataset_folder_path)
if not os.path.exists(mimic_dataset_path):
    os.mkdir(mimic_dataset_path)


In [ ]:
def save_to_json(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file)

def load_from_json(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def zip_directory(source_dir, output_zip):
    shutil.make_archive(output_zip, 'zip', source_dir)



In [ ]:
# load data from dataset file

# with open("mimicbertSum.json", "r") as f:
#     data = json.load(f)

# data = data[:2464]
# print(len(data))
# extractives = [item["topK_importance"] for item in data]
# summaries = [item["summary"] for item in data]
# # print(" ".join(extractives[0]))

# extractives = [" ".join(extract) for extract in extractives]
# summaries = [" ".join(summary) for summary in summaries]

# print('finished')

### Split data and save to seperate file

In [ ]:
# from sklearn.model_selection import train_test_split

# train_extractives, test_extractives, train_summaries, test_summaries = train_test_split(extractives, summaries, test_size=0.1, random_state=42)
# train_extractives, val_extractives, train_summaries, val_summaries = train_test_split(train_extractives, train_summaries, test_size=0.1, random_state=42)


# dataset_dir = mimic_dataset_path

# train_extractives = save_to_json(train_extractives, os.path.join(dataset_dir,'train_extractives.json'))
# # test_extractives = save_to_json(test_extractives, os.path.join(dataset_dir, 'test_extractives.json'))
# train_summaries = save_to_json(train_summaries, os.path.join(dataset_dir, 'train_summaries.json'))
# # test_summaries = save_to_json(test_summaries, os.path.join(dataset_dir, 'test_summaries.json'))
# val_extractives = save_to_json(val_extractives, os.path.join(dataset_dir, 'val_extractives.json'))
# val_summaries = save_to_json(val_summaries, os.path.join(dataset_dir, 'val_summaries.json'))

# print('finished')


In [ ]:
# zip_directory(dataset_folder_path, dataset_folder_path+'-700-end')
# # files.download(dataset_folder_path+'.zip')

# zip_directory(mimic_dataset_path, mimic_dataset_path)
# files.download(mimic_dataset_path+'.zip')

### read data from seperated file

In [ ]:
# read from json
dataset_dir = mimic_dataset_path
!ls mimicDataset
train_extractives = load_from_json(os.path.join(dataset_dir,'train_extractives.json'))
test_extractives = load_from_json(os.path.join(dataset_dir, 'test_extractives.json'))
train_summaries = load_from_json(os.path.join(dataset_dir, 'train_summaries.json'))
test_summaries = load_from_json(os.path.join(dataset_dir, 'test_summaries.json'))
val_extractives = load_from_json(os.path.join(dataset_dir, 'val_extractives.json'))
val_summaries = load_from_json(os.path.join(dataset_dir, 'val_summaries.json'))

test_extractives.json  train_extractives.json  val_extractives.json
test_summaries.json    train_summaries.json    val_summaries.json


In [ ]:
print(len(train_extractives))
print(len(val_extractives))
print(len(test_extractives))

2000
223
241


# Train and Validate Different HPs

In [ ]:
# !pip install transformers datasets sentencepiece
# !pip install --upgrade accelerate

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from transformers import PegasusXForConditionalGeneration
import torch


"""
some part of this block modified from https://gist.github.com/jiahao87/50cec29725824da7ff6dd9314b53c4b3
"""

# class PegasusDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item["labels"] = torch.tensor(self.labels["input_ids"][idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])

# class PegasusDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels, input_ids=None):
#         self.encodings = encodings
#         self.labels = labels
#         self.input_ids = input_ids

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item["labels"] = torch.tensor(self.labels["input_ids"][idx])
#         if self.input_ids is not None:
#             item["input_ids"] = torch.tensor(self.input_ids[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, input_ids=None):
        self.encodings = encodings
        self.labels = labels
        self.input_ids = input_ids

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels["input_ids"][idx]
        if self.input_ids is not None:
            item["input_ids"] = self.input_ids[idx]
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


'''
make ready-to-use dataset
'''      
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):

  # tokenizer = PegasusTokenizer.from_pretrained(model_name)
  # tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")
  print(tokenizer)
  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  # def tokenize_data(texts, labels):
  #   encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
  #   decodings = tokenizer(labels, truncation=True, padding=True, max_length=512)
  #   tokenized_dataset = PegasusDataset(encodings, decodings)
  #   return tokenized_dataset
  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding='longest', max_length=1024, return_tensors="pt")
    decodings = tokenizer(labels, truncation=True, padding='longest', max_length=1024, return_tensors="pt")
    tokenized_dataset = PegasusDataset(encodings, decodings, input_ids=encodings["input_ids"])
    return tokenized_dataset


  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer


print('prepare_data finished runing!')

prepare_data finished runing!


In [ ]:
"""
some part of this block modified from https://gist.github.com/jiahao87/50cec29725824da7ff6dd9314b53c4b3
"""


def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  # model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
  # model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
  model = PegasusXForConditionalGeneration.from_pretrained(model_name).to(torch_device) # for Pegasus-x
  print(model)
    
  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=30,           # total number of training epochs
      per_device_train_batch_size=7,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=7,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=4,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      learning_rate=3e-5,              # set beginning lr 
      # lr_scheduler_type='cosine_with_restarts', # set scheduler type
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.08,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
      log_level='info',
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=100,           # total number of training epochs
      per_device_train_batch_size=8,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.005,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer


print('prepare_fine_tuning finished runing!')

prepare_fine_tuning finished runing!


In [ ]:
if __name__=='__main__':
  # use XSum dataset as example, with first 1000 docs as training data
  # from datasets import load_dataset
  # dataset = load_dataset("xsum")
  # train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]
  
  # use Pegasus Large model as base for fine-tuning
  print('start train!')
  model_name = 'google/pegasus-x-large'
  # _, _, test_dataset, _ = prepare_data(model_name, train_extractives, train_summaries, val_extractives, val_summaries, test_extractives, test_summaries)
  train_dataset, val_dataset, test_dataset, tokenizer = prepare_data(model_name, train_extractives, train_summaries, val_extractives, val_summaries)
  trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset, freeze_encoder=False)
  trainer.train()

  # write training log to checkpoint
  log_file_path = './logs/training_logs.txt'
  with open(log_file_path, 'w') as log_file:
    log_file.write(f"Training Arguments:\n{json.dumps(trainer.args.to_dict(), indent=2)}\n\n")
    log_file.write("Training and Validation Losses:\n")
    with open(trainer.args.logging_dir + '/info.log', 'r') as log_info:
      log_file.writelines(log_info.readlines())

start train!


loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--pegasus-x-large/snapshots/ff38c5db1f5b97b923dbecdaf58b15c8e213a27a/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--pegasus-x-large/snapshots/ff38c5db1f5b97b923dbecdaf58b15c8e213a27a/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--pegasus-x-large/snapshots/ff38c5db1f5b97b923dbecdaf58b15c8e213a27a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--pegasus-x-large/snapshots/ff38c5db1f5b97b923dbecdaf58b15c8e213a27a/tokenizer_config.json


PegasusTokenizerFast(name_or_path='google/pegasus-x-large', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '

## TEST USE

In [ ]:
# # 1. Load the checkpoint model
# checkpoint_path = "./results/checkpoint-500"
# eval_model = PegasusXForConditionalGeneration.from_pretrained(checkpoint_path)

# # 2. Load the tokenizer
# eval_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")
    
# # Check if GPU is available and set the device accordingly
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move the model to the device
# model.to(device)

# # Modify the generate_summary function to use GPU
# def generate_summary(model, tokenizer, text, max_length=4096):
#     inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True)

#     # Move input tensors to the device
#     inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

#     summary_ids = model.generate(**inputs, no_repeat_ngram_size=3)
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary





In [ ]:
# # Test the model on your input text
# input_text = """
# Most movements are comprised of sequences. From the complex routines that gymnasts perform to intricate dance numbers to simply reaching for an object, our movements are comprised of sequences of movements that are learned through practice. The cerebellum has been long implicated in learning and execution of accurate movements (Doyon et al., 1997; Shin and Ivry, 2003; Lehéricy et al., 2005; Krupa et al., 1993; Lisberger, 1994; Diener and Dichgans, 1992). Movement sequences as well as multi-joint movements are particularly sensitive to cerebellar dysfunction (Shin and Ivry, 2003; Diener and Dichgans, 1992; Doyon et al., 2002). For example, one of the hallmark deficits of cerebellar pathology is dysdiadochokinesia (Diener and Dichgans, 1992) – an inability to perform a rapid alternating sequence of movements. Patients with cerebellar lesions display severe deficits (Doyon et al., 1997) in sequence learning or are unable to learn sequences at all (Shin and Ivry, 2003), even with modest impairments in learning of single-component, directly cued movements (Spencer and Ivry, 2009). Studies of sequence learning in nonhuman primates (Desmurget and Turner, 2010; Rünger et al., 2013) demonstrated that with repeated training components of movement in the sequence start to be initiated predictively, before the arrival of sensory cue. Since one of the fundamental properties of cerebellar learning is the ability to learn a predictive response (Marr, 1969; Bastian, 2006; Shadmehr et al., 2010; Therrien and Bastian, 2015), these results also indirectly imply a strong cerebellar contribution to sequence learning. Although results of these studies suggest cerebellar involvement in the learning and execution of movement sequences, most of what we know about cerebellar mechanisms of learning comes from studies utilizing single-component movements. These include adaptation of the vestibule-ocular reflex, adaptation of saccadic and smooth pursuit eye movements and conditioning of eyelid responses. In principle, the cerebellar mechanisms underlying movement sequences could be different from, or at least somewhat different from those mediating single-component movements. If so, these mechanisms are largely unknown. In contrast, we tested the hypothesis that a simple elaboration of cerebellar mechanisms that mediate learning single-component movements is sufficient to explain cerebellar learning and implementation of movement sequences. Three possible (but not mutually exclusive) ways of implementing cerebellar learning of movement sequences are illustrated in Figure 1. With the first possibility a specific external cue is associated with a specific movement component in the sequence (Figure 1A). The second option is a variant of the first one. Here only single external cue is present, but it persists in time through the whole sequence, so that different movement components are elicited by signals associated with different times during the cue (Figure 1B). Finally, Figure 1C illustrates a possibility where feedback signals from one movement component are used to learn the next component. The design of most experiments does not permit distinguishing between these possibilities. For example, a number of studies (Choi and Moore, 2003; Moore and Choi, 1997; Freeman et al., 2003; Halverson et al., 2015; Jirenhed et al., 2017) used eyelid conditioning to train subjects to respond with a sequence of two eyelid responses. In these experiments, however, the external cue either explicitly extended through the whole sequence
# """
# print(input_text)
# summary = generate_summary(eval_model, eval_tokenizer, input_text)
# print(summary)

In [ ]:
# !nproc
# !tar cf - results | pigz -0 -p 12 > results.tar.gz


# Evaluation on Held-Out Test Set

In [ ]:
# %%capture
# !pip3 install datasets
# !pip3 install rouge_score

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 1.5 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 22.7 MB/s eta 0:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24956 sha256=e6d5db37fc3c1c78863f964ea36a17591550c08d1c98f32d337da81b8db71f9a
  Stored in directory: /root/.cache/pip/wheels/fa/9b/18/7518ccfe3107aca76ec573a2fa1264e00efb0f3168a66f09c2
Successfully built rouge-score


In [ ]:
from datasets import load_metric, list_metrics
print(list_metrics())

/tmp/ipykernel_11011/1100646279.py:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  print(list_metrics())


['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'character', 'charcut_mt', 'chrf', 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'mape', 'mase', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'nist_mt', 'pearsonr', 'perplexity', 'poseval', 'precision', 'r_squared', 'recall', 'rl_reliability', 'roc_auc', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'smape', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s', 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics', 'Asmedeus/perplexity', 'BucketHeadP65/confusion_matrix', 'BucketHeadP65/roc_curve', 'Drunper/metrica_tesi', 'Felipehonorato/eer', 'GMFTBY/dailydialog_evaluate', 'GMFTBY/dailydialogevaluate', 'JP-SystemsX/nDCG', 'Josh98/nl2bash_m', 'KevinSpaghetti/accuracyk', 'Muennighoff/code_eval', 'NCSOFT/harim_plus', 'NikitaMartynov/spell-check-metric', 'Nima

In [ ]:
# model_name = 'google/bigbird-pegasus-large-pubmed'
# model_path = "checkpoint-3500"
# model_path = "checkpoint-4000"
checkpoint_path = "./results/checkpoint-2500"


torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_model = PegasusXForConditionalGeneration.from_pretrained(checkpoint_path).to(torch_device)

# Put the model in evaluation mode
eval_model.eval()

print("finished this!")

loading configuration file ./results/checkpoint-2500/config.json
Model config PegasusXConfig {
  "_name_or_path": "google/pegasus-x-large",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusXForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "block_size": 512,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,

finished this!


Let's take 2 samples and verify the predictions to be sure everything works as expected

In [ ]:
print(len(test_dataset))

241


Finally, we can evaluate the predictions using the *rouge* metric.

In [ ]:
# Read and parse the JSON file into a Python list
with open("mimicDataset/test_extractives.json", "r") as file:
    data = json.load(file)
print(len(data))
# Calculate the total number of tokens (words) in the list
total_tokens = 0
for string in data:
    tokens = string.split()  # Split the string into tokens by whitespace
    total_tokens += len(tokens)

# Calculate the average number of tokens per string
average_tokens_ext = total_tokens / len(data)
print(f"Average number of test extractives tokens: {average_tokens_ext}")

with open("mimicDataset/test_summaries.json", "r") as file:
    data = json.load(file)

# Calculate the total number of tokens (words) in the list
total_tokens = 0
for string in data:
    tokens = string.split()  # Split the string into tokens by whitespace
    total_tokens += len(tokens)

# Calculate the average number of tokens per string
average_tokens_sums = total_tokens / len(data)
print(f"Average number of test summary tokens: {average_tokens_sums}")

241
Average number of test extractives tokens: 820.5311203319502
Average number of test summary tokens: 383.09128630705396


In [ ]:
import numpy as np
# Generate summaries for the test dataset
generated_summaries = []
for num, example in enumerate(test_dataset):
    print(num)
    input_ids = example['input_ids'].unsqueeze(0)
    input_ids = input_ids.to(eval_model.device)
    attention_mask = example['attention_mask'].unsqueeze(0)
    attention_mask = attention_mask.to(eval_model.device)
    summary_ids = eval_model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=round(average_tokens_sums), num_beams=3, length_penalty=0.8)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)
    # print(summary)
np.save('gen_sums_finetuned_bertsum.npy', np.array(generated_summaries))

print("finished generate!")

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 16384,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.27.4"
}



0


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 16384,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.27.4"
}



1


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 16384,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.27.4"
}



...

238


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 16384,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.27.4"
}



239


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 16384,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.27.4"
}



240
finished generate!


### Rouge

In [ ]:
print("load Rouge matric")
rouge = load_metric("rouge")
print(len(test_summaries))
# Create a list to store the reference summaries
gen_sums = np.load('gen_sums_finetuned_bertsum.npy')
reference_summaries = test_summaries

# Compute ROUGE-L scores
rouge_results = rouge.compute(predictions=gen_sums, references=reference_summaries, rouge_types=["rougeL", "rouge1", "rouge2"], use_aggregator=True)

# Access the ROUGE-L score
rouge_l_score = rouge_results["rougeL"].mid.fmeasure
rouge_1_score = rouge_results["rouge1"].mid.fmeasure
rouge_2_score = rouge_results["rouge2"].mid.fmeasure



load Rouge matric
241


In [ ]:
# Print the ROUGE-L score
print("Rouge scores for finetuned finetuned Pegasus-x-large on 2k andy data")
print("ROUGE-L Aggregated F1 Score:", rouge_l_score)
print("ROUGE-1 Aggregated F1 Score:", rouge_1_score)
print("ROUGE-2 Aggregated F1 Score:", rouge_2_score)

Rouge scores for finetuned finetuned Pegasus-x-large on 2k andy data
ROUGE-L Aggregated F1 Score: 0.18643294804538302
ROUGE-1 Aggregated F1 Score: 0.33473625347582453
ROUGE-2 Aggregated F1 Score: 0.0684724793511705


### BLEU(NOT USE)

In [ ]:
# # BLEU
# import nltk
# nltk.download('punkt')
# bleu = load_metric("bleu")

# gen_sums = np.load('gen_sums_bertsum.npy').reshape(-1,1).tolist()
# ref_sums = np.array(test_summaries)
# ref_sums = np.expand_dims(np.expand_dims(ref_sums, axis=1), axis=1).tolist()
# # print(len(gen_sums[0][0]))
# # print(len(ref_sums[0][0][0]))
# # print(gen_sums[0][0])
# # print(ref_sums[0][0][0])

# def custom_tokenizer(text):
#     return nltk.word_tokenize(text)

# # Tokenize the predictions and references
# # tokenized_preds = [[custom_tokenizer(pred) for pred in pred] for pred in gen_sums]
# # tokenized_references = [[[custom_tokenizer(ref) for ref in refs] for refs in group] for group in ref_sums]

# # print(tokenized_preds)

# bleu_results = bleu.compute(predictions=gen_sums, references=ref_sums, max_order=1, tokenizer=word_tokenize)

# # Access the BLEU score
# # bleu_score = bleu_results["bleu"]

# # print("BLEU Score n=1:", bleu_score)

In [ ]:
# !pip install bert_score

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 61 kB 1.5 MB/s eta 0:00:01


### BERTScore

In [ ]:
# Load the BERTScore metric
print("Load the BERTScore metric")
bert_score_metric = load_metric("bertscore")

gen_sums = np.load('gen_sums_finetuned_bertsum-xLarge1.npy')
ref_sums = test_summaries

# Compute BERTScore using the loaded metric
print("Compute BERTScore using the loaded metric")
bertscore_results = bert_score_metric.compute(predictions=gen_sums, references=ref_sums, lang="en")

print("compute BERTScore finished!")

Load the BERTScore metric
Compute BERTScore using the loaded metric


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/716877d372b884cad6d419d828bac6c85b3b18d9/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub

compute BERTScore finished!


In [ ]:
avg_bert_score = np.array(bertscore_results['f1']).mean()
print("Avg BertScore-F1 of finetuned Pegasus-x-large on andy data 2k: ", avg_bert_score)

Avg BertScore-F1 of finetuned Pegasus-x-large on andy data 2k:  0.8180586609108319


### Meteor

In [ ]:
print("load meteor!")
meteor = load_metric("meteor")

gen_sums = np.load('gen_sums_finetuned_bertsum.npy')
ref_sums = test_summaries

print("Compute METEOR scores")
# Compute METEOR scores
meteor_scores = meteor.compute(predictions=gen_sums, references=ref_sums, alpha=0.001, beta=30, gamma=0.001)

# Access the METEOR score
meteor_score = meteor_scores["meteor"]

# Print the METEOR score
print("METEOR of finetuned Pegasus-x-large on andy data 2k: ", meteor_score)

load meteor!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


KeyboardInterrupt: 